# Fall 2024 Data Science Track: Week 5 - Unsupervised Learning

## Packages, Packages, Packages!

Import *all* the things here! You need: `matplotlib`, `networkx`, `numpy`, and `pandas`―and also `ast.literal_eval` to correctly deserialize two columns in the `rules.tsv.xz` file.

If you got more stuff you want to use, add them here too. 🙂

In [2]:
# Don’t worry about this. This is needed to interpret the Python code that is embedded in the data set. You only need it literally in the very next code cell and nowhere else. 
from ast import literal_eval

# The rest is just the stuff from the lecture.
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

# Instacart Association Rules

## Introduction

With the packages out of the way, now you will be working with the Instacart association rules data set, mined from the [Instacart Market Basket Analysis data set](https://www.kaggle.com/c/instacart-market-basket-analysis/data) on Kaggle. [The script](https://github.com/LiKenun/shopping-assistant/blob/main/api/preprocess_instacart_market_basket_analysis_data.py) that does it and the instructions to run it can be found in my [Shopping Assistant Project](https://github.com/LiKenun/shopping-assistant) repository.

## Load the Data

This code has already been pre-written, simply because there are a few quirks which require converters to ensure the correct deserialization of some columns.

In [61]:
rules_data_path = 'data/rules.tsv.xz'                       # You do not need to decompress this yourself. Pandas understands how to read compressed data.

df_rules = pd.read_csv(rules_data_path,
                       sep='\t',
                       quoting=3,                           # This disables interpretation of quotes by Pandas itself, because both single and double quotes will be resolved by literal_eval.
                       converters={
                           'consequent_item': literal_eval,
                           'antecedent_items': literal_eval # This reads something like ["Grandma's 8\" Chip Cookies", '6" Apple Pie'] into a list, so you will get a column where each individual cell is a list.
                       },
                       low_memory=True)                     # For Chris

But just *how* many rules were just loaded‽

In [62]:
# Show the list of column names and the number of rules.

#list(df_rules.col)
#len(df_rules)
df_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   consequent_item    1048575 non-null  object
 1   transaction_count  1048575 non-null  int64 
 2   item_set_count     1048575 non-null  int64 
 3   antecedent_count   1048575 non-null  int64 
 4   consequent_count   1048575 non-null  int64 
 5   antecedent_items   1048575 non-null  object
dtypes: int64(4), object(2)
memory usage: 48.0+ MB


## Metrics

Compute the support, confidence, and lift of each rule.

* The rule’s *support* tells you how frequently the set of items appears in the dataset. It’s important to prune infrequent sets from further consideration.
    * The simple definition: $$P(A \cap B)$$
    * `= item_set_count / transaction_count`
* The rule’s *confidence* tells you how often a the rule is true. Divide the support for the set of items by the support for just the antecedents. Rules which are not true very often are also pruned.
    * The simple definition: $$\frac{P(A \cap B)}{P(A)}$$
    * `= item_set_count / transaction_count / (antecedent_count / transaction_count)`
    * `= item_set_count / antecedent_count`
* The rule’s *lift* tells you how much more likely the consequent is, given the antecedents, compared to its baseline probability. Divide the support for the set of items by both the support of the antecedents and consequent. Equivalently, divide the confidence by the support of the consequent.
    * The simple definition: $$\frac{P(A \cap B)}{P(A) \cdot P(B)}$$
    * `= item_set_count / transaction_count / (antecedent_count / transaction_count * (consequent_count / transaction_count))`
    * `= item_set_count / antecedent_count / (consequent_count / transaction_count)`
    * `= item_set_count * transaction_count / (antecedent_count * consequent_count)`

In [63]:
# Add new columns support, confidence, and lift to df_rules. And show the first 50 rules.

df_rules['support'] = df_rules['item_set_count'] / df_rules['transaction_count']
df_rules['confidence'] = df_rules['item_set_count'] / df_rules['antecedent_count']
df_rules['lift'] = df_rules['item_set_count'] * df_rules['transaction_count'] / (df_rules['antecedent_count'] * df_rules['consequent_count'])
df_rules.head(50)

,consequent_item,transaction_count,item_set_count,antecedent_count,consequent_count,antecedent_items,support,confidence,lift
0,Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.000030,0.821138,21465.598514
1,Total 2% Lowfat Greek Strained Yogurt With Blueberry,3346083,101,128,123,"[Fat Free Strawberry Yogurt, Total 0% Raspberry Yogurt, Total 0% with Honey Nonfat Greek Strained Yogurt, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.000030,0.789062,21465.598514
2,Total 0% with Honey Nonfat Greek Strained Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.000030,0.821138,21465.598514
3,Total 0% Raspberry Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.000030,0.821138,21465.598514
4,Pineapple Yogurt 2%,3346083,101,128,123,"[Fat Free Strawberry Yogurt, Total 0% Raspberry Yogurt, Total 0% with Honey Nonfat Greek Strained Yogurt, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.000030,0.789062,21465.598514
5,Fat Free Strawberry Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.000030,0.821138,21465.598514
6,Fat Free Blueberry Yogurt,3346083,101,128,123,"[Fat Free Strawberry Yogurt, Total 0% Raspberry Yogurt, Total 0% with Honey Nonfat Greek Strained Yogurt, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.000030,0.789062,21465.598514
7,Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt,3346083,101,147,117,"[Blackberry Yogurt, Fat Free Strawberry Yogurt, Total 2% All Natural Greek Strained Yogurt with Honey, Total 2% Greek Strained Yogurt with Cherry 5.3 oz]",0.000030,0.687075,19649.653061
8,Total 2% Lowfat Greek Strained Yogurt With Blueberry,3346083,101,147,117,"[Blackberry Yogurt, Fat Free Strawberry Yogurt, Total 2% All Natural Greek Strained Yogurt with Honey, Total 2% Greek Strained Yogurt with Cherry 5.3 oz]",0.000030,0.687075,19649.653061
9,Total 2% Greek Strained Yogurt with Cherry 5.3 oz,3346083,101,117,147,"[Pineapple Yogurt 2%, Total 0% Raspberry Yogurt, Total 2% Lowfat Greek Strained Yogurt With Blueberry, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.000030,0.863248,19649.653061


The yogurts have got some insane lift (*over 9,000*). Why do you think that might be?

*(Write your answer here.)*

In [64]:
# Query the rule set if you have to to find out more.

pd.set_option('display.max_colwidth', None)
condition = df_rules['lift'] > 9000
df_rules[condition].head()

# A possible explanation for such a high count for lift could be that some items in the antecedent items + the consequent item could be sold in a bundle/variety pack of yogurt. THe high lift makes sense because these 
# items that are often sold in a bundle would appear together. 

,consequent_item,transaction_count,item_set_count,antecedent_count,consequent_count,antecedent_items,support,confidence,lift
0,Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.00003,0.821138,21465.598514
1,Total 2% Lowfat Greek Strained Yogurt With Blueberry,3346083,101,128,123,"[Fat Free Strawberry Yogurt, Total 0% Raspberry Yogurt, Total 0% with Honey Nonfat Greek Strained Yogurt, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.00003,0.789062,21465.598514
2,Total 0% with Honey Nonfat Greek Strained Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.00003,0.821138,21465.598514
3,Total 0% Raspberry Yogurt,3346083,101,123,128,"[Fat Free Blueberry Yogurt, Pineapple Yogurt 2%, Total 2% Lowfat Greek Strained Yogurt With Blueberry]",0.00003,0.821138,21465.598514
4,Pineapple Yogurt 2%,3346083,101,128,123,"[Fat Free Strawberry Yogurt, Total 0% Raspberry Yogurt, Total 0% with Honey Nonfat Greek Strained Yogurt, Total 2% with Raspberry Pomegranate Lowfat Greek Strained Yogurt]",0.00003,0.789062,21465.598514


## Network Visualization for Consequents with Single Antecedents

Let’s now visualize a small subset of 1,000,000+ rules. First, filter the rule set for the following to whittle it down to something more manageable:

1. The rule must have exactly `1` antecedent item. (There should be 38,684 such rules.)
2. The lift must be between `5` and `20`. (There should be 1,596 such rules, including the prior criterion.)
3. Either the antecedent or consequent of the rule must contain `'Hummus'`, but not both. (This should get you down to 26 rules.)
    * Convert the antecedents `list`-typed column to a `str`-typed column (`antecedent_item`) since there will only be a single antecedent in the subset.
    * Replace any item containing `'Hummus'` to just `'Hummus'`. This will make the visualization more readable later.

Hint: your code may run more efficiently if you re-order certain processing steps.

Assign the subset to `df_rules_subset`.

In [65]:
# Define df_rules_subset.

condition_1 = df_rules['antecedent_items'].apply(len) == 1
condition_2 = df_rules['lift'].between(5,20)
len(df_rules[condition_1 & condition_2]) # 1596
df_rules['antecedent_item'] = df_rules['antecedent_items'].astype(str)
df_rules['antecedent_item'] = df_rules['antecedent_item'].replace('.*Hummus.*', 'Hummus', regex=True)
df_rules['antecedent_items'] = df_rules['antecedent_item']
df_rules_subset = df_rules[condition_1 & condition_2]
df_rules_subset

,consequent_item,transaction_count,item_set_count,antecedent_count,consequent_count,antecedent_items,support,confidence,lift,antecedent_item
373979,2% Reduced Fat Milk,3346083,471,2048,38631,['2% Chocolate Milk'],0.000141,0.229980,19.920109,['2% Chocolate Milk']
374026,Soda,3346083,125,564,37298,['Peppermint Patties'],0.000037,0.221631,19.883007,['Peppermint Patties']
374036,Total 2% All Natural Greek Strained Yogurt with Honey,3346083,1050,7413,23844,['Fat Free Blueberry Yogurt'],0.000314,0.141643,19.877094,['Fat Free Blueberry Yogurt']
374081,Organic Tomato Paste,3346083,610,5608,18336,['Crushed Tomatoes With Basil'],0.000182,0.108773,19.849700,['Crushed Tomatoes With Basil']
374095,Total 2% Greek Strained Yogurt with Cherry 5.3 oz,3346083,1126,10162,18692,['Fat Free Strawberry Yogurt'],0.000337,0.110805,19.835362,['Fat Free Strawberry Yogurt']
...,...,...,...,...,...,...,...,...,...,...
587817,Carrots,3346083,1216,10787,75233,['Organic Low Sodium Vegetable Broth'],0.000363,0.112728,5.013733,['Organic Low Sodium Vegetable Broth']
588017,Organic Red Onion,3346083,339,3197,70804,['Organic Mint Bunch'],0.000101,0.106037,5.011134,['Organic Mint Bunch']
588271,Michigan Organic Kale,3346083,286,2727,70076,['Organic Soy Tempeh'],0.000085,0.104877,5.007815,['Organic Soy Tempeh']
588826,Organic Small Bunch Celery,3346083,8985,85005,70714,['Organic Cucumber'],0.002685,0.105700,5.001553,['Organic Cucumber']


Build a network `graph_rules_subset` from the association rules subset.

In [ ]:
# Define graph_rules_subset, add the graph’s edges, and plot it. You may need a large figure size, smaller node size, and smaller font size.

graph_rules_subset = nx.MultiDiGraph()
graph_rules_subset.add_edges_from(
    # Something goes here.
)

# Then render the graph.

What can you tell about people who buy hummus?

*(Write your answer here.)*

## Make a Prediction

Given that the basket of items contains the following items, use the full set of association rules to predict the next 20 most likely items (consequents) that the person will add to the basket in descending order of lift:

* `'Orange Bell Pepper'`
* `'Organic Red Bell Pepper'`

Hint: a single item in the basket may be a better predictor of some consequents than both items considered together. You must consider both or either, but not neither.

In [ ]:
basket = {'Orange Bell Pepper', 'Organic Red Bell Pepper'}

df_rules[# A few conditions go here which takes care of all the following cases:
         # * Just Orange Bell Pepper
         # * Just Organic Red Bell Pepper
         # * Both Orange Bell Pepper and Organic Red Bell Pepper
         # You can do it using just 2 conditions. :)
         ] \
        .sort_values('lift', ascending=False) \
        .head(20)

## Bonus: Other Interesting Findings

Find and share something else interesting about these association rules. It can be a graph, table, or some other format that illustrates your point.